# Set Covering Problem
## Parsing the problem

Each problem instant is represented by
- set of elements A of size m
- set of subsets F of A of size n
- weights of subsets


In [2]:
import os
import re
import numpy as np

class SCPInstance:
    def __init__(self, index, folder="SCP-Instances", sol_file="Solutions.txt"):
        self.folder = folder
        self.sol_file = sol_file
        self.index = index

        self.filename = self._get_filename(index)
        self.name = self.filename.replace(".txt", "")

        self.path = os.path.join(folder, self.filename)

        self.m, self.n, self.costs, self.set_to_attrs, self.attr_to_sets = self._load_instance()
        self.opt_value = self._load_opt_value()

    # ------------------------------------------------------------
    # 🔹 Load the instance from file
    # ------------------------------------------------------------
    def _get_filename(self, index):
        files = sorted(
            [f for f in os.listdir(self.folder)
             if f.lower().startswith("scp") and f.lower().endswith(".txt")]
        )
        if not files:
            raise FileNotFoundError(f"No SCP files found in {self.folder}.")
        if isinstance(index, int):
            if index >= len(files):
                raise IndexError(f"Index {index} out of range (found {len(files)} files).")
            filename = files[index]
        else:
            raise TypeError("Index must be an integer.")
        return filename

    def _load_instance(self):
        with open(self.path, "r") as f:
            data = list(map(int, f.read().split()))

        m, n = data[0], data[1]
        costs = data[2:2 + n]

        set_to_attrs = [set() for _ in range(n)]
        attr_to_sets = [set() for _ in range(m)]

        idx = 2 + n
        for attr in range(m):
            k_i = data[idx]
            idx += 1
            airplanes = data[idx:idx + k_i]
            idx += k_i
            for j in airplanes:
                set_to_attrs[j - 1].add(attr)
                attr_to_sets[attr].add(j - 1)

        return m, n, costs, set_to_attrs, attr_to_sets

    # ------------------------------------------------------------
    # 🔹 Load known optimal value
    # ------------------------------------------------------------
    def _load_opt_value(self):
        base = self.filename.lower().replace("scp", "").replace(".txt", "")
        sol_id = f"{base[0].upper()}.{base[1:]}" if base[0].isalpha() else f"{base[0]}.{base[1:]}"
        opt_value = None

        if os.path.exists(self.sol_file):
            with open(self.sol_file, "r") as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) >= 2 and parts[0].upper() == sol_id:
                        opt_value = float(parts[1])
                        break
        else:
            print(f"⚠️ Solutions file '{self.sol_file}' not found in current directory.")

        return opt_value

    # ------------------------------------------------------------
    # 🔹 Pretty print summary
    # ------------------------------------------------------------
    def summary(self, max_show=4):
        print("=" * 70)
        print(f"📘 Instance: {self.filename}")
        print(f"  Attributes (m): {self.m}")
        print(f"  Airplanes (n):  {self.n}")
        print(f"  Known optimal cost: {self.opt_value if self.opt_value else 'Unknown'}")
        print("-" * 70)
        print("Costs sample:", self.costs[:6], "..." if len(self.costs) > 10 else "")
        print("-" * 70)
        print("Example coverage:")
        for i in range(min(max_show, self.m)):
            print(f"  Attribute {i}: covered by {list(self.attr_to_sets[i])[:8]}")
        print("=" * 70)

In [3]:
inst = SCPInstance(3)
inst.summary()

📘 Instance: scp45.txt
  Attributes (m): 200
  Airplanes (n):  1000
  Known optimal cost: 512.0
----------------------------------------------------------------------
Costs sample: [1, 1, 1, 1, 1, 1] ...
----------------------------------------------------------------------
Example coverage:
  Attribute 0: covered by [774, 7, 779, 656, 274, 541, 798, 290]
  Attribute 1: covered by [130, 261, 393, 400, 657, 532, 155, 412]
  Attribute 2: covered by [2, 34, 35, 930, 166, 302, 814, 833]
  Attribute 3: covered by [260, 518, 647, 649, 905, 532, 32, 160]


# Solution representation

A solution is represented by a binary vector of size n, where each position i indicates whether subset i is included in the cover (1) or not (0).

In [4]:
class SCPSolution:
    def __init__(self, instance):
        """
        Initialize an empty solution for a given SCPInstance.
        """
        self.instance = instance  # reference to the problem data
        self.selected = set()     # chosen airplanes
        self.covered = np.zeros(instance.m, dtype=int)  # number of sets covering each attribute
        self.cost = 0.0

    # ------------------------------------------------------------
    # 🔹 Add airplane j
    # ------------------------------------------------------------
    def add(self, j):
        if j in self.selected:
            return
        self.selected.add(j)
        self.cost += self.instance.costs[j]
        for a in self.instance.set_to_attrs[j]:
            self.covered[a] += 1

    # ------------------------------------------------------------
    # 🔹 Remove airplane j
    # ------------------------------------------------------------
    def remove(self, j):
        if j not in self.selected:
            return
        self.selected.remove(j)
        self.cost -= self.instance.costs[j]
        for a in self.instance.set_to_attrs[j]:
            self.covered[a] -= 1

    # ------------------------------------------------------------
    # 🔹 Check feasibility
    # ------------------------------------------------------------
    def is_feasible(self):
        return np.all(self.covered > 0)

    # ------------------------------------------------------------
    # 🔹 Get uncovered attributes
    # ------------------------------------------------------------
    def uncovered_attributes(self):
        return np.where(self.covered == 0)[0].tolist()

    def prune(self):
        to_remove =[]
        for j in list(self.selected):
            self.remove(j)
            if self.is_feasible():
                to_remove.append(j)
            else:
                self.add(j)
    
    def prune_by_cost(self):
        """
        Redundancy elimination (RE) procedure:
        Try removing sets in decreasing cost order.
        A set is removed if the solution remains feasible.
        """
        # Sort selected sets by descending cost
        sets_sorted = sorted(list(self.selected), key=lambda j: self.instance.costs[j], reverse=True)

        for j in sets_sorted:
            # Temporarily remove the set
            self.remove(j)

            # Check feasibility (all elements must still be covered)
            if not self.is_feasible():
                # Restore if it breaks coverage
                self.add(j)


    # ------------------------------------------------------------
    # 🔹 Pretty print
    # ------------------------------------------------------------
    def summary(self, max_show=10):
        print("=" * 60)
        print(f"✈️  SCP Solution Summary")
        print(f"  Selected airplanes: {len(self.selected)} ")
        print(f"  Total cost: {self.cost}")
        print(f"  Feasible: {self.is_feasible()}")
        uncovered = self.uncovered_attributes()
        print(f"  Uncovered attributes: {len(uncovered)}")
        print("-" * 60)
        print(f"Selected (sample): {sorted(list(self.selected))[:max_show]}")
        if uncovered:
            print(f"Uncovered (sample): {uncovered[:max_show]}")
        print("=" * 60)
        print("\n")


In [5]:
inst = SCPInstance(3)
sol = SCPSolution(inst)

sol.add(5)
sol.add(10)
sol.add(200)
sol.remove(10)

sol.summary()
sol.prune()
sol.summary()


✈️  SCP Solution Summary
  Selected airplanes: 2 
  Total cost: 20.0
  Feasible: False
  Uncovered attributes: 196
------------------------------------------------------------
Selected (sample): [5, 200]
Uncovered (sample): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


✈️  SCP Solution Summary
  Selected airplanes: 2 
  Total cost: 20.0
  Feasible: False
  Uncovered attributes: 196
------------------------------------------------------------
Selected (sample): [5, 200]
Uncovered (sample): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]




# Constructive heuristics

## Greedy First Fit

### The next one that covers the first uncovered attribute

h() = j that covers the first uncovered attribute i

A simple greedy constructive heuristic
- Look into the attributes that are not covered
- Find the first airplane that covers the first uncovered attribute
- Add that airplane to the solution


- Pruning later

Characteristics:
- Deterministic
- Myopic

In [6]:
def greedy_first_fit(instance):
    """
    Simple greedy constructive heuristic.
    - Look into uncovered attributes
    - Find the first airplane that covers the first uncovered one
    - Add that airplane to the solution
    """
    sol = SCPSolution(instance)

    while not sol.is_feasible():
        # Get uncovered attributes
        uncovered = sol.uncovered_attributes()
        if not uncovered:
            break

        attr = uncovered[0]  # pick the first uncovered attribute
        candidates = sorted(instance.attr_to_sets[attr])

        if not candidates:
            raise ValueError(f"No airplane covers attribute {attr}!")

        chosen = candidates[0]  # take the first covering airplane
        sol.add(chosen)

    return sol


In [7]:
inst = SCPInstance(0)  # load first file in folder
sol = greedy_first_fit(inst)
sol.summary()
sol.prune()
sol.summary()


✈️  SCP Solution Summary
  Selected airplanes: 81 
  Total cost: 616.0
  Feasible: True
  Uncovered attributes: 0
------------------------------------------------------------
Selected (sample): [0, 1, 2, 4, 5, 6, 7, 9, 10, 11]


✈️  SCP Solution Summary
  Selected airplanes: 70 
  Total cost: 572.0
  Feasible: True
  Uncovered attributes: 0
------------------------------------------------------------
Selected (sample): [0, 1, 2, 4, 5, 6, 7, 9, 10, 12]




## Greedy Cost-Efficient

### The minimum cost per uncovered attributes covered

h() = airplane j that minimizes ( cost_j /number_of_new_uncovered_attributes_covered_by_j )








In [8]:
def greedy_cost_efficiency(instance):
    """
    Greedy constructive heuristic:
    Choose the airplane that minimizes (cost / number of new uncovered attributes covered).
    """
    sol = SCPSolution(instance)

    while not sol.is_feasible():
        best_j = None
        best_ratio = float("inf")

        for j in range(instance.n):
            if j in sol.selected:
                continue

            # number of uncovered attributes this airplane would cover
            new_cover = sum(sol.covered[a] == 0 for a in instance.set_to_attrs[j])
            if new_cover == 0:
                continue  # useless airplane

            ratio = instance.costs[j] / new_cover
            if ratio < best_ratio:
                best_ratio = ratio
                best_j = j

        if best_j is None:
            raise ValueError("No airplane can cover remaining attributes!")

        sol.add(best_j)

    return sol


In [9]:
inst = SCPInstance(0)
sol = greedy_cost_efficiency(inst)
sol.summary()
sol.prune()
sol.summary()


✈️  SCP Solution Summary
  Selected airplanes: 81 
  Total cost: 582.0
  Feasible: True
  Uncovered attributes: 0
------------------------------------------------------------
Selected (sample): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


✈️  SCP Solution Summary
  Selected airplanes: 66 
  Total cost: 533.0
  Feasible: True
  Uncovered attributes: 0
------------------------------------------------------------
Selected (sample): [0, 2, 3, 4, 5, 7, 9, 10, 12, 14]




## Greedy randomized adaptive search procedure (GRASP) constructive heuristic
 
- I implemented just the constructive part
- Same as greedy cost efficient, but instead of picking the best, it picks randomly from the best candidates
- alpha is a parameter that controls the greediness vs randomness
- alpha = 0 is purely greedy
- alpha = 1 is purely random


In [10]:
import random
random.seed(42)  # reproducible results

def greedy_randomized_adaptive(instance, alpha=0.1):
    """
    CH3: Greedy Randomized Adaptive Constructive Heuristic (GRASP-style)
    Builds a feasible cover using Restricted Candidate List (RCL).
    alpha ∈ [0,1]: controls greediness vs randomness.
    alpha = 0 -> purely greedy (same as CH2).
    """
    sol = SCPSolution(instance)

    while not sol.is_feasible():
        efficiencies = []

        for j in range(instance.n):
            if j in sol.selected:
                continue

            # number of uncovered elements this set covers
            new_cover = sum(sol.covered[a] == 0 for a in instance.set_to_attrs[j])
            if new_cover == 0:
                continue

            eff = instance.costs[j] / new_cover
            efficiencies.append((eff, j))

        if not efficiencies:
            raise ValueError("No remaining set can cover uncovered elements!")

        efficiencies.sort(key=lambda x: x[0])
        best, worst = efficiencies[0][0], efficiencies[-1][0]

        if alpha == 0:
            # Pure greedy selection (deterministic)
            chosen = efficiencies[0][1]
        else:
            threshold = best + alpha * (worst - best)
            RCL = [j for eff, j in efficiencies if eff <= threshold]
            chosen = random.choice(RCL)

        sol.add(chosen)

    return sol


# Look ahead heuristics

In [23]:
def greedy_with_future_lb(instance, lam=0.3, mode="min"):
    """
    Non-myopic constructive heuristic for Set Cover.
    Combines greedy cost-efficiency with a look-ahead term that estimates
    the remaining cost of uncovered elements.

    Parameters
    ----------
    instance : SCPInstance
        Contains:
          - n: number of sets
          - m: number of elements
          - costs[j]: cost of set j
          - set_to_attrs[j]: list of elements covered by set j
          - attr_to_sets[a]: list of sets that cover element a
    lam : float in [0,1]
        Trade-off between current and future cost importance.
    mode : str
        Defines how the future cost h is estimated:
          - "min"        : element-wise cheapest-set lower bound
          - "weighted"   : weighted coverage (scarcity-based)
          - "fractional" : fractional weighted coverage
    """
    sol = SCPSolution(instance)

    # Precompute cheapest covering set per element
    cmin = [min(instance.costs[j] for j in instance.attr_to_sets[a]) for a in range(instance.m)]

    # Average cost used in weighted modes
    c_avg = sum(instance.costs) / instance.n

    while not sol.is_feasible():
        uncovered = [a for a in range(instance.m) if sol.covered[a] == 0]
        best_j, best_score = None, float("inf")

        for j in range(instance.n):
            if j in sol.selected:
                continue

            new_elems = [a for a in instance.set_to_attrs[j] if sol.covered[a] == 0]
            if not new_elems:
                continue

            # --- Immediate term: current greedy cost-efficiency ---
            g = instance.costs[j] / len(new_elems)

            # --- Future cost term: estimate of remaining uncovered after adding j ---
            remaining = [a for a in uncovered if a not in new_elems]

            if mode == "min":
                # Element-wise minimum cost lower bound
                h = sum(cmin[a] for a in remaining)

            elif mode == "weighted":
                # Weighted coverage: harder elements contribute more
                h = sum(c_avg / len(instance.attr_to_sets[a]) for a in remaining)

            elif mode == "fractional":
                # Fractional version: redundancy- and cost-aware
                h = 0.0
                for a in remaining:
                    denom = sum(1 / instance.costs[k] for k in instance.attr_to_sets[a])
                    h += 1 / denom if denom > 0 else float("inf")

            else:
                raise ValueError(f"Unknown mode '{mode}'")

            score = g + lam * h

            if score < best_score:
                best_score, best_j = score, j

        if best_j is None:
            raise ValueError("No remaining set can cover uncovered elements!")

        sol.add(best_j)

    return sol


# Running all instances


In [26]:
def solve_instance(index, heuristic_func, name):
    """Runs a given heuristic (with and without pruning) for one instance."""
    inst = SCPInstance(index)

    # --- Run heuristic ---
    sol = heuristic_func(inst)
    cost_before = sol.cost

    # --- Apply pruning (redundancy elimination) ---
    sol.prune_by_cost()
    cost_after = sol.cost

    # --- Get optimum value ---
    opt = inst.opt_value

    return inst.name, name, cost_before, cost_after, opt


def run_all_instances(folder="SCP-Instances", num_instances=0):
    """
    Runs all heuristics on SCP instances.
    - num_instances = 0 → run all instances
    - num_instances = k → run only first k instances
    """
    files = sorted([f for f in os.listdir(folder) if f.lower().startswith("scp")])
    if num_instances > 0:
        files = files[:num_instances]

    results = []

    # Define your constructive heuristics
    heuristics = [
        ("CH1", greedy_first_fit),
        ("CH2", greedy_cost_efficiency),
        #("CH3", greedy_randomized_adaptive),
        ("CH4", greedy_with_future_lb),  
    ]

    # ANSI colors
    GREEN = "\033[92m"   # optimal match
    YELLOW = "\033[93m"  # best among your heuristics
    RESET = "\033[0m"

    for i in range(len(files)):
        inst_results = {"Instance": SCPInstance(i).name}
        opt_value = None

        # Run each heuristic
        for name, func in heuristics:
            inst_name, hname, cost_before, cost_after, opt = solve_instance(i, func, name)
            inst_results[f"{name}_Before"] = cost_before
            inst_results[f"{name}_After"] = cost_after
            opt_value = opt

        inst_results["Opt"] = opt_value
        results.append(inst_results)

    # --- Print results in table ---
    headers = [
        "Instance", "Opt",
        "CH1_Before", "CH1_After",
        "CH2_Before", "CH2_After",
        #"CH3_Before", "CH3_After",
        "CH4_Before", "CH4_After"
    ]

    print("\nSummary of results:")
    print(" | ".join(f"{h:<12}" for h in headers))
    print("-" * 110)

    for r in results:
        # Find best (lowest) cost among all CHs (before/after)
        all_costs = [r[h] for h in headers[2:] if isinstance(r.get(h), (int, float))]
        best_mine = min(all_costs) if all_costs else None

        row_parts = []
        for h in headers:
            val = r.get(h, "?")
            if isinstance(val, (int, float)):
                # --- Coloring logic ---
                if abs(val - r["Opt"]) < 1e-9:           # exact optimum
                    colored_val = f"{GREEN}{val:>12.2f}{RESET}"
                elif best_mine is not None and abs(val - best_mine) < 1e-9:
                    colored_val = f"{YELLOW}{val:>12.2f}{RESET}"
                else:
                    colored_val = f"{val:>12.2f}"
                row_parts.append(colored_val)
            else:
                row_parts.append(f"{val:<12}")
        print(" | ".join(row_parts))


In [27]:
run_all_instances(num_instances=10)


Summary of results:
Instance     | Opt          | CH1_Before   | CH1_After    | CH2_Before   | CH2_After    | CH4_Before   | CH4_After   
--------------------------------------------------------------------------------------------------------------
scp42        |       512.00 |       616.00 |       572.00 |       582.00 |       529.00 |       574.00 |       573.00
scp43        |       516.00 |       589.00 |       537.00 |       598.00 |       537.00 |       662.00 |       662.00
scp44        |       494.00 |       585.00 |       529.00 |       548.00 |       506.00 |       639.00 |       639.00
scp45        |       512.00 |       624.00 |       527.00 |       577.00 |       518.00 |       603.00 |       601.00
scp46        |       560.00 |       655.00 |       610.00 |       615.00 |       594.00 |       781.00 |       781.00
scp47        |       430.00 |       529.00 |       455.00 |       476.00 |       447.00 |       576.00 |       568.00
scp48        |       492.00 |       560.00